In [1]:
from nnunetv2.evaluation.evaluate_predictions import compute_metrics_on_folder, labels_to_list_of_regions
from nnunetv2.imageio.nibabel_reader_writer import NibabelIOWithReorient
import json

folder_ref = '/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_raw/Dataset901_mosaic_abdomen_pathologies/labelsTr'
model_folder = '/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_results/Dataset901_mosaic_abdomen_pathologies/nnUNetTrainer_MOSAIC_250_QuarterLR_NoMirroring__MOSAIC_nnUNetPlannerResEncL_stdres_NoRsmp_24G__3d_fullres_stdres_NoRsmp'
folder_pred = f'{model_folder}/fold_0/validation'
output_file = f'{folder_pred}/summary.json'
dataset_json = f'{model_folder}/dataset.json'
image_reader_writer = NibabelIOWithReorient()
file_ending = '.nii.gz'
ignore_label = None
num_processes = 18

with open(dataset_json) as f:
    data = json.loads(f.read())

# regions = labels_to_list_of_regions(list(data['labels'].values())[1:])

regions = list(data['labels'].values())[1:]

compute_metrics_on_folder(folder_ref, folder_pred, output_file, image_reader_writer, file_ending, regions, ignore_label,
                            num_processes)

{'metric_per_case': [{'reference_file': '/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_raw/Dataset901_mosaic_abdomen_pathologies/labelsTr/adrenal_Ki67_Seg_002_native.nii.gz',
   'prediction_file': '/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_results/Dataset901_mosaic_abdomen_pathologies/nnUNetTrainer_MOSAIC_250_QuarterLR_NoMirroring__MOSAIC_nnUNetPlannerResEncL_stdres_NoRsmp_24G__3d_fullres_stdres_NoRsmp/fold_0/validation/adrenal_Ki67_Seg_002_native.nii.gz',
   'metrics': {1: {'Dice': 0.97540177174473,
     'IoU': 0.9519846363639395,
     'FP': 18081,
     'TP': 1427639,
     'FN': 53925,
     'TN': 43300355,
     'n_pred': 1445720,
     'n_ref': 1481564},
    2: {'Dice': 0.9331659342958778,
     'IoU': 0.874705790051781,
     'FP': 205,
     'TP': 11149,
     'FN': 1392,
     'TN': 44787254,
     'n_pred': 11354,
     'n_ref': 12541},
    3: {'Dice': 0.9596499347835233,
     'IoU': 0.9224298309471811,
     'FP': 2919,
     'TP': 79828,
     'FN': 3794,
     'TN': 44713459,

### Binary evaluation

In [ ]:
import json
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

results_folder = '/home/eolika/Desktop/BlueMind/nnUNet/trains/nnUNet_results/Dataset901_mosaic_abdominal_pathology/nnUNetTrainer_MOSAIC_500_QuarterLR_NoMirroring__Mosaic_nnUNetPlannerResEncL_stdres_NoRsmp_24G__3d_fullres_stdres_NoRsmp'
folds = ['fold_0']

with open(f"{results_folder}/dataset.json", 'r') as f:
    labels_data = json.load(f)['labels']

# Remove background label
del labels_data['background']

def calculate_binary_sensitivity_and_specificity(label, summary):
    TP = FP = FN = TN = 0

    # Iterate through each case in the summary
    for case in summary['metric_per_case']:
        n_ref = case['metrics'][str(label)]['n_ref']
        n_pred = case['metrics'][str(label)]['n_pred']
        
        if n_ref > 0 and n_pred > 0:
            TP += 1
        elif n_ref > 0 and n_pred == 0:
            FN += 1
        elif n_ref == 0 and n_pred > 0:
            FP += 1
        elif n_ref == 0 and n_pred == 0:
            TN += 1

    sensitivity = TP / (TP + FN) if (TP + FN) > 0 else 0
    specificity = TN / (TN + FP) if (TN + FP) > 0 else 0

    return TP, FP, FN, TN, sensitivity, specificity

# Initialize a dictionary to store results for each label
results = {label: {'TP': 0, 'FP': 0, 'FN': 0, 'TN': 0, 'sensitivity': 0, 'specificity': 0} for label in labels_data.values()}

# Iterate through each fold and calculate metrics
for fold in folds:
    if os.path.exists(f"{results_folder}/{fold}/validation/summary.json"):
        with open(f"{results_folder}/{fold}/validation/summary.json", 'r') as f:
            summary = json.load(f)
        
        for label_name, label in labels_data.items():
            TP, FP, FN, TN, sensitivity, specificity = calculate_binary_sensitivity_and_specificity(label, summary)
            results[label]['TP'] += TP
            results[label]['FP'] += FP
            results[label]['FN'] += FN
            results[label]['TN'] += TN

# Calculate average sensitivity and specificity for each label
for label in results.keys():
    TP = results[label]['TP']
    FP = results[label]['FP']
    FN = results[label]['FN']
    TN = results[label]['TN']
    
    results[label]['sensitivity'] = TP / (TP + FN) if (TP + FN) > 0 else 0
    results[label]['specificity'] = TN / (TN + FP) if (TN + FP) > 0 else 0

# Create confusion matrices for each label
confusion_matrices = {}

for label in results.keys():
    confusion_matrix = np.array([
        [results[label]['TN'], results[label]['FP']],
        [results[label]['FN'], results[label]['TP']]
    ])
    confusion_matrices[label] = confusion_matrix

if not os.path.exists(f"{results_folder}/confusion_matrix"):
    os.mkdir(f"{results_folder}/confusion_matrix")

# Plot the confusion matrix for each label
for label_name, label in labels_data.items():
    df_cm = pd.DataFrame(confusion_matrices[label], index=["Actual Negative", "Actual Positive"],
                         columns=["Predicted Negative", "Predicted Positive"])

    plt.figure(figsize=(8, 6))
    sns.heatmap(df_cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.title(f'Confusion Matrix for {label_name}')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.savefig(f"{results_folder}/confusion_matrix/{label_name}.png")
    plt.show()

# Print average sensitivity and specificity for each label
for label_name, label in labels_data.items():
    print(f"Label: {label_name}")
    print(f"Sensitivity: {results[label]['sensitivity']:.4f}")
    print(f"Specificity: {results[label]['specificity']:.4f}")
    print()